In [1]:
import sys
sys.path.append("/Users/Daniel/Rigetti/pyquil") # CHANGE THIS TO YOUR ABSOLUTE PATH TO THE PYQUIL FOLDER

In [2]:
import pyquil

In [3]:
from pyquil.quil import Program 
from pyquil.gates import *
from pyquil.api import QVMConnection
from pyquil.paulis import ID, sX, sY, sZ

In [4]:
import numpy as np
import scipy
from scipy.linalg import expm, sinm, cosm, logm

In [5]:
K = np.array([[0.5065, 0.2425], 
              [0.2425, 0.4935]])
gamma = 1000
F = K + 1/gamma*np.eye(2)
print(F)

[[0.5075 0.2425]
 [0.2425 0.4945]]


In [77]:
class Inverse():
    def __init__(self, F, program, qvm, theta0, theta1, theta2):
        self.F = F
        self.program = program
        self.qvm = qvm
        self.first, self.second, self.y, self.anc = (0,1,2,3)
        
        self.theta0 = theta0
        self.theta1 = theta1
        self.theta2 = theta2

#     def expmatrix(self, val,matrix):  
#         return expm(val*1j*self.F)
    
    def controlMake(self, M):
        """Creates control gates from M.
        param:
            M: (matrix) to control
        returns:
            (matrix) controlled."""
        zero = np.zeros((2,2))
        I = np.eye(2)
        top = np.concatenate((I, zero), axis = 1)
        bottom = np.concatenate((zero, M), axis = 1)
        res = np.concatenate((top, bottom), axis = 0)
        return res
    
    def CRY(self, angle):
        """Creates control RY gate.
        param:
            angle: (float) by how much to rotate
        returns:
            matrix."""
        Y = np.array([[0, -1j], [1j, 0]])
        expY = expm(-angle/2. * 1j * Y)
        return self.controlMake(expY)
    
    def CCRY(self, angle):
        Y = np.array([[0, -1j], [1j, 0]])
        expY = expm(-angle/2. * 1j * Y)
        
        ccry = np.eye(8, dtype='cfloat')
        ccry[6][6] = expY[0][0]
        ccry[6][7] = expY[0][1]
        ccry[7][6] = expY[1][0]
        ccry[7][7] = expY[1][1]

        return ccry
    
    def NCCRY(self, angle):
        Y = np.array([[0, -1j], [1j, 0]])
        expY = expm(-angle/2. * 1j * Y)
        
        cncry = np.eye(8, dtype='cfloat')
        cncry[2][2] = expY[0][0]
        cncry[2][3] = expY[0][1]
        cncry[3][2] = expY[1][0]
        cncry[3][3] = expY[1][1]
        
        return cncry
        
    def HGate(self):
        return self.controlMake(np.sqrt(0.5) * np.array([[1,1], [1,-1]]))
    
    
    def main(self):
        
        # =============================================
        # Inversion 
        # =============================================
        # add hadamards
        self.program += H(self.first)
        self.program += H(self.second)
        
        # add the exponent gates
        expF = expm(np.pi*1j*self.F)
        expF = self.controlMake(expF)
        self.program = self.program.defgate("expF", expF)
        self.program.inst(("expF", self.first, self.y))
        
        expFhalf = expm(np.pi/2.*1j*self.F)
        expFhalf = self.controlMake(expFhalf)
        self.program = self.program.defgate("expFhalf", expFhalf)
        self.program.inst(("expFhalf", self.second, self.y))
        
        self.program += SWAP(self.first, self.second)
        self.program += H(self.second)
        
        #S inverse
        self.program += CPHASE(-np.pi/2, self.second, self.first) # right order of qubits?
        
        self.program += H(self.first)
        
        CRYpi4 = self.CRY(np.pi/4.)
        self.program = self.program.defgate("CRYpi4", CRYpi4)
        self.program.inst(("CRYpi4", self.second, self.anc))
        
        CRYpi8 = self.CRY(np.pi/8.)
        self.program = self.program.defgate("CRYpi8", CRYpi8)
        self.program.inst(("CRYpi8", self.first, self.anc))  
        
        self.program += H(self.first)
        
        self.program += CPHASE(np.pi/2, self.second, self.first) # right order of qubits?
        
        self.program += H(self.second)
        
        self.program += SWAP(self.first, self.second)
        
        minusExpFhalf = expm(-np.pi/2.*1j*self.F)
        minusExpFhalf = self.controlMake(minusExpFhalf)
        self.program = self.program.defgate("minusExpFhalf", minusExpFhalf)
        self.program.inst(("minusExpFhalf", self.second, self.y))
        
        minusExpF = expm(-np.pi*1j*self.F)
        minusExpF = self.controlMake(minusExpF)
        self.program = self.program.defgate("minusExpF", minusExpF)
        self.program.inst(("minusExpF", self.second, self.y))
        
        self.program += H(self.first)
        self.program += H(self.second)
        
        # =============================================
        # Training Data Orcale 
        # =============================================
        
        theta1Gate = self.NCCRY(self.theta1)
        self.program = self.program.defgate("theta1Gate", theta1Gate)
        self.program.inst(("theta1Gate", self.anc, self.y, self.second))
        
        theta2Gate = self.CCRY(self.theta2)
        self.program = self.program.defgate("theta2Gate", theta2Gate)
        self.program.inst(("theta2Gate", self.anc, self.y, self.second))
        
        # =============================================
        # U_x0
        # =============================================
        
        theta0Gate = self.CRY(-self.theta0)
        self.program = self.program.defgate("theta0Gate", theta0Gate)
        self.program.inst(("theta0Gate", self.anc, self.second))
        
        CH = self.HGate()
        self.program = self.program.defgate("CH", CH)
        self.program.inst(("CH", self.anc, self.y))
        
        # =============================================
        # MEASUREMENT
        # =============================================
        
        self.program += SWAP(self.first, self.anc)
        
        self.program += MEASURE(self.anc, 0)
        
        return self.qvm.wavefunction(self.program)
        

        
        
        


        


In [62]:
def classify(wavefunction):
    print(np.real(wavefunction.amplitudes[9]))
    if np.real(wavefunction.amplitudes[9]) > 0:
        return 9
    else:
        return 6

In [63]:
# 9-train
inverse = Inverse(F, Program(), QVMConnection(), np.arctan(1/(0.354/0.935)), np.arctan(1/(0.987/0.159)), np.arctan(1/(0.354/0.935)))
wavefunction = inverse.main()
print(wavefunction)
print("Classified as: ", classify(wavefunction))

-0.4753547997j|1000> + (0.125199337-0.0877551274j)|1001> + (0.6396312069+0j)|1010> + (0.3026399391+0.0720921489j)|1011> + (-0.0361663209-0.0108206482j)|1100> + (0.125199337-0.1210068459j)|1101> + (0.4519027209-0.0008659896j)|1110> + (0.0599080091+0.0720921489j)|1111>
0.12519933702287936
Classified as:  9


In [64]:
# 6-train
inverse = Inverse(F, Program(), QVMConnection(), np.arctan(1/(0.987/0.159)), np.arctan(1/(0.987/0.159)), np.arctan(1/(0.354/0.935)))
wavefunction = inverse.main()
print(wavefunction)
print("Classified as: ", classify(wavefunction))

-0.4753547997j|1000> + (-0.0432099514-0.1120622394j)|1001> + (0.6396312069+0j)|1010> + (0.3246516699+0.0184470816j)|1011> + (-0.0361663209-0.0108206482j)|1100> + (0.0783604159-0.1408428878j)|1101> + (0.4519027209-0.0008659896j)|1110> + (0.1145577967+0.0017932214j)|1111>
-0.04320995140727065
Classified as:  6


In [65]:
# 9-test1
inverse = Inverse(F, Program(), QVMConnection(), np.arctan(1/(0.147/0.989)), np.arctan(1/(0.987/0.159)), np.arctan(1/(0.354/0.935)))
wavefunction = inverse.main()
print(wavefunction)
print("Classified as: ", classify(wavefunction))

-0.4753547997j|1000> + (0.1568582054-0.0795388313j)|1001> + (0.6396312069+0j)|1010> + (0.2875088+0.0810667295j)|1011> + (-0.0361663209-0.0108206482j)|1100> + (0.1308899502-0.1125997123j)|1101> + (0.4519027209-0.0008659896j)|1110> + (0.0461699521+0.084624107j)|1111>
0.15685820539207548
Classified as:  9


In [68]:
# 6-test1
inverse = Inverse(F, Program(), QVMConnection(), np.arctan(1/(0.997/-0.072)), np.arctan(1/(0.987/0.159)), np.arctan(1/(0.354/0.935)))
wavefunction = inverse.main()
print(wavefunction)
print("Classified as: ", classify(wavefunction))

-0.4753547997j|1000> + (-0.0804651475-0.1134436957j)|1001> + (0.6396312069+0j)|1010> + (0.3174762459+0.0053636015j)|1011> + (-0.0361663209-0.0108206482j)|1100> + (0.0645863323-0.1401052599j)|1101> + (0.4519027209-0.0008659896j)|1110> + (0.1228513298-0.0145069239j)|1111>
-0.08046514748359092
Classified as:  6


In [70]:
# 9-test2
inverse = Inverse(F, Program(), QVMConnection(), np.arctan(1/(0.338/0.941)), np.arctan(1/(0.987/0.159)), np.arctan(1/(0.354/0.935)))
wavefunction = inverse.main()
print(wavefunction)
print("Classified as: ", classify(wavefunction))

-0.4753547997j|1000> + (0.1277809595-0.0871358635j)|1001> + (0.6396312069+0j)|1010> + (0.3015590043+0.0728394235j)|1011> + (-0.0361663209-0.0108206482j)|1100> + (0.1257067064-0.1203863679j)|1101> + (0.4519027209-0.0008659896j)|1110> + (0.0588359373+0.0731235743j)|1111>
0.1277809595169964
Classified as:  9


In [74]:
# 6-test2
inverse = Inverse(F, Program(), QVMConnection(), np.arctan(1/(0.999/0.025)), np.arctan(1/(0.987/0.159)), np.arctan(1/(0.354/0.935)))
wavefunction = inverse.main()
print(wavefunction)
print("Classified as: ", classify(wavefunction))

-0.4753547997j|1000> + (-0.0649611026-0.1130496608j)|1001> + (0.6396312069+0j)|1010> + (0.3210075729+0.0108634483j)|1011> + (-0.0361663209-0.0108206482j)|1100> + (0.0704729939-0.140644249j)|1101> + (0.4519027209-0.0008659896j)|1110> + (0.1195717387-0.0076895969j)|1111>
-0.06496110255767043
Classified as:  6


In [85]:
# PROCESSING
from matplotlib.pyplot import imshow
from PIL import Image
import numpy as np

def downsize(img):
    return img.resize((28, 28), Image.ANTIALIAS)

def get_HR(img):
    """Dark pixels in left half / Dark pixels in right half."""
    left_total, right_total = 0, 0
    width, height = img.size
    
    for x in range(width):
        for y in range(height):
            r, g, b, a = img.getpixel((x, y))
            brightness = (r + g + b) / 3
            if x < (width / 2):
                left_total += 1 - (brightness / 255)
            else:
                right_total += 1 - (brightness / 255)
    
    return left_total / right_total

def get_VR(img):
    """Dark pixels in top half / Dark pixels in bottom half."""
    top_total, bottom_total = 0, 0
    width, height = img.size
    
    for x in range(width):
        for y in range(height):
            r, g, b, a = img.getpixel((x, y))
            brightness = (r + g + b) / 3
            if y < (height / 2):
                bottom_total += 1 - (brightness / 255)
            else:
                top_total += 1 - (brightness / 255)
    
    return bottom_total / top_total

def load_image(path):
    return Image.open(path)

def full_process(img):
    """Features HR, VR as an array."""
    img = downsize(img)
    return [get_HR(img), get_VR(img) * 0.6]

In [86]:
# 9-processed
img_9 = load_image("9.png")
features_9 = full_process(img_9)
inverse = Inverse(F, Program(), QVMConnection(), np.arctan(1/(features_9[0]/features_9[1])), np.arctan(1/(0.987/0.159)), np.arctan(1/(0.354/0.935)))
wavefunction = inverse.main()
print(wavefunction)
print("Classified as: ", classify(wavefunction))

-0.4753547997j|1000> + (0.0549996493-0.1015524026j)|1001> + (0.6396312069+0j)|1010> + (0.3228635087+0.0508463356j)|1011> + (-0.0361663209-0.0108206482j)|1100> + (0.1088051206-0.1339769048j)|1101> + (0.4519027209-0.0008659896j)|1110> + (0.0861701183+0.0434755525j)|1111>
0.05499964934641784
Classified as:  9


In [87]:
# 6-processed
img_6 = load_image("6.png")
features_6 = full_process(img_6)
print(features_6)
inverse = Inverse(F, Program(), QVMConnection(), np.arctan(1/(features_6[0]/features_6[1])), np.arctan(1/(0.987/0.159)), np.arctan(1/(0.354/0.935)))
wavefunction = inverse.main()
print(wavefunction)
print("Classified as: ", classify(wavefunction))

[1.3039439374185129, 0.46847939535720756]
-0.4753547997j|1000> + (-0.0130058491-0.1098764418j)|1001> + (0.6396312069+0j)|1010> + (0.3272562522+0.0287299121j)|1011> + (-0.0361663209-0.0108206482j)|1100> + (0.0886173248-0.1400736943j)|1101> + (0.4519027209-0.0008659896j)|1110> + (0.1068214084+0.0148086073j)|1111>
-0.013005849064087957
Classified as:  6
